### 加载Overlay

Overlay模块封装了ARM CPU与FPGA的PL部分进行交互的接口。

- 我们可以通过简单的`Overlay()`方法将刚才生成的硬件设计加载到PL上
- 通过`overlay.fir_wrap_0`语句，我们可以通过访问的Python对象的形式来与IP交互

In [1]:
from pynq import Overlay
overlay = Overlay("./0520icecs.bit")
icecs = overlay.DSVS_search_0

### 分配内存供IP使用
`pynq.allocate`函数用于为PL中的IP分配可以使用的内存空间。
- 在PL中的IP访问DRAM之前，必须为其保留一些内存供IP使用，分配大小与地址
- 我们分别为输入、输出和权重三个部分分配内存，数据类型为int32
- `pynq.allocate`会分配物理上的连续内存，并返回一个`pynq.Buffer`表示已经分配缓冲区的对象

In [2]:
from pynq import allocate
query_len = 29184
#query_len = 288
data_len = 24511520
data_index_len = 6127880
voxel_len = 1303400
sub_flag_len = 1303400
sub_len = 1303400
result_len = 7296
#result_len = 72
nearest_len = 7296
#nearest_len = 72
query_buffer = allocate(shape=(query_len,), dtype='f4')
data_buffer = allocate(shape=(data_len,), dtype='f4')
data_index_buffer = allocate(shape=(data_index_len,), dtype='i4')
voxel_buffer = allocate(shape=(voxel_len,), dtype='i4')
sub_flag_buffer = allocate(shape=(sub_flag_len,), dtype='i4')
sub_buffer = allocate(shape=(sub_len,), dtype='i4')
result_buffer = allocate(shape=(result_len,), dtype='i4')
nearest_buffer = allocate(shape=(nearest_len,), dtype='f4')
#coef_buffer = allocate(shape=(99,), dtype='i4')

将python的本地内存中的音频数据和系数数据，复制到我们刚分配的内存中。

In [3]:
f0 = open('ordered_query.txt','r')
#a = f0.readline()
#a = a.split()
#print(a[0])

In [4]:

for i in range (7296):
    a = f0.readline()
    a = a.split()
    query_buffer[4*i] = a[0]
    query_buffer[4*i+1] = a[1]
    query_buffer[4*i+2] = a[2]
    query_buffer[4*i+3] = 4*i+3
    #query_buffer[4*i] = 4*i+3
    #query_buffer[4*i+1] = a[2]
    #query_buffer[4*i+2] = a[1]
    #query_buffer[4*i+3] = a[0]

f1 = open('ordered_ref.txt','r')
f2 = open('original_dataset_index.txt','r')

for i in range (6127880):
    b = f1.readline()
    b = b.split()
    data_buffer[4*i] = b[0]
    data_buffer[4*i+1] = b[1]
    data_buffer[4*i+2] = b[2]
    data_buffer[4*i+3] = 4*i+3
    #data_buffer[4*i] = 4*i+3
    #data_buffer[4*i+1] = b[2]
    #data_buffer[4*i+2] = b[1]
    #data_buffer[4*i+3] = b[0]
    c = f2.readline()
    c = c.split()
    data_index_buffer[i] = c[0]

f3 = open('voxel_first_index_PL.txt','r')
f4 = open('sub_voxel_first_index_PL.txt','r')
f5 = open('sub_voxel_flag_index_PL.txt','r')

for i in range (1303400):
    d = f3.readline()
    d = d.split()
    voxel_buffer[i] = d[0]
    e = f4.readline()
    e = e.split()
    sub_buffer[i] = e[0]
    f = f5.readline()
    f = f.split()
    sub_flag_buffer[i] = f[0]

In [5]:
#np.copyto(input_buffer, np.int32(aud_in))
#np.copyto(coef_buffer, hpf_coeffs_hw)

In [6]:
data_buffer[15900:16001]

PynqBuffer([-4.6915e+01, -8.5197e+01,  6.8420e+00,  1.5903e+04,
            -4.7307e+01, -8.5177e+01,  7.1600e+00,  1.5907e+04,
            -4.7291e+01, -8.5883e+01,  7.1620e+00,  1.5911e+04,
            -4.7234e+01, -8.5709e+01,  7.4370e+00,  1.5915e+04,
            -4.7195e+01, -8.6050e+01,  7.2630e+00,  1.5919e+04,
            -4.7195e+01, -8.5272e+01,  7.3390e+00,  1.5923e+04,
            -4.7178e+01, -8.5934e+01,  7.3110e+00,  1.5927e+04,
            -4.7178e+01, -8.5249e+01,  7.1060e+00,  1.5931e+04,
            -4.7156e+01, -8.5779e+01,  7.1760e+00,  1.5935e+04,
            -4.7130e+01, -8.6073e+01,  7.6970e+00,  1.5939e+04,
            -4.7115e+01, -8.6075e+01,  7.3360e+00,  1.5943e+04,
            -4.7089e+01, -8.5179e+01,  7.5910e+00,  1.5947e+04,
            -4.7070e+01, -8.5556e+01,  7.6830e+00,  1.5951e+04,
            -4.7072e+01, -8.5667e+01,  7.3900e+00,  1.5955e+04,
            -4.7061e+01, -8.5383e+01,  7.8920e+00,  1.5959e+04,
            -4.7060e+01, -8.5601e+01,  7

我们可以看到，缓冲区本质也是numpy数组，但是提供了一些物理地址属性。

In [7]:
data_buffer[990:1010]

PynqBuffer([ 8.9700e-01,  9.9100e+02, -5.7710e+01,  2.8720e+01,
             8.8000e-01,  9.9500e+02, -5.7744e+01,  2.8163e+01,
             3.0330e+00,  9.9900e+02, -5.7726e+01,  2.8097e+01,
             2.9930e+00,  1.0030e+03, -5.7710e+01,  2.7947e+01,
             2.9900e+00,  1.0070e+03, -5.7713e+01,  2.8040e+01],
           dtype=float32)

In [8]:
data_buffer.physical_address

1613758464

In [9]:
data_buffer[31900:32000]

PynqBuffer([-4.7122e+01, -6.7352e+01,  5.1420e+00,  3.1903e+04,
            -4.7103e+01, -6.7483e+01,  5.2990e+00,  3.1907e+04,
            -4.7068e+01, -6.7302e+01,  5.3700e+00,  3.1911e+04,
            -4.7255e+01, -6.7152e+01,  5.7190e+00,  3.1915e+04,
            -4.7241e+01, -6.7270e+01,  5.9060e+00,  3.1919e+04,
            -4.7244e+01, -6.7259e+01,  5.7360e+00,  3.1923e+04,
            -4.7217e+01, -6.7467e+01,  5.5980e+00,  3.1927e+04,
            -4.7221e+01, -6.7204e+01,  5.5480e+00,  3.1931e+04,
            -4.7208e+01, -6.7525e+01,  5.8200e+00,  3.1935e+04,
            -4.7206e+01, -6.7496e+01,  5.6790e+00,  3.1939e+04,
            -4.7187e+01, -6.7541e+01,  5.7670e+00,  3.1943e+04,
            -4.7155e+01, -6.7475e+01,  5.7350e+00,  3.1947e+04,
            -4.7150e+01, -6.7221e+01,  5.6110e+00,  3.1951e+04,
            -4.7108e+01, -6.7327e+01,  5.6630e+00,  3.1955e+04,
            -4.7045e+01, -6.7569e+01,  5.6950e+00,  3.1959e+04,
            -4.7011e+01, -6.7187e+01,  5

In [10]:
data_buffer[6127800:6127880]

PynqBuffer([ 1.450400e+01, -5.979700e+01,  1.840000e+00,  6.127803e+06,
             1.450900e+01, -5.998400e+01,  1.841000e+00,  6.127807e+06,
             1.452100e+01, -5.995800e+01,  1.845000e+00,  6.127811e+06,
             1.455400e+01, -5.961900e+01,  1.602000e+00,  6.127815e+06,
             1.455600e+01, -5.960600e+01,  1.780000e+00,  6.127819e+06,
             1.458100e+01, -5.991400e+01,  1.822000e+00,  6.127823e+06,
             1.458100e+01, -5.999000e+01,  1.846000e+00,  6.127827e+06,
             1.459800e+01, -5.999700e+01,  1.976000e+00,  6.127831e+06,
             1.459500e+01, -5.996600e+01,  1.845000e+00,  6.127835e+06,
             1.460700e+01, -6.006000e+01,  1.982000e+00,  6.127839e+06,
             1.461000e+01, -5.963400e+01,  1.746000e+00,  6.127843e+06,
             1.461100e+01, -5.960500e+01,  1.795000e+00,  6.127847e+06,
             1.463500e+01, -5.997500e+01,  1.971000e+00,  6.127851e+06,
             1.463900e+01, -5.960000e+01,  1.717000e+00,  6.1278

In [11]:
data_buffer[0:100]

PynqBuffer([-57.176, -54.138,   5.663,   3.   , -57.18 , -54.158,   5.67 ,
              7.   , -57.14 , -54.122,   5.468,  11.   , -57.042, -54.408,
              5.671,  15.   , -57.027, -54.397,   5.572,  19.   , -56.964,
            -54.217,   5.605,  23.   , -57.055, -54.408,   6.085,  27.   ,
            -57.149, -53.411,   3.935,  31.   , -57.283, -53.515,   5.687,
             35.   , -57.269, -54.008,   5.688,  39.   , -57.22 , -53.785,
              5.72 ,  43.   , -57.207, -53.272,   5.718,  47.   , -57.167,
            -53.934,   5.733,  51.   , -57.137, -53.643,   5.467,  55.   ,
            -57.134, -53.661,   5.683,  59.   , -57.126, -53.511,   5.559,
             63.   , -57.123, -53.995,   5.561,  67.   , -57.118, -53.734,
              5.612,  71.   , -57.119, -53.557,   5.628,  75.   , -57.115,
            -53.81 ,   5.589,  79.   , -57.07 , -53.914,   5.57 ,  83.   ,
            -57.033, -53.871,   5.537,  87.   , -57.165, -52.59 ,   3.934,
             91.   , -57.

In [12]:
data_buffer[24511200:24511520]

PynqBuffer([ 9.8347000e+01, -4.9051998e+01,  2.8720000e+00,
             2.4511204e+07,  9.9459000e+01, -1.0570800e+02,
             7.0359998e+00,  2.4511208e+07,  9.9573997e+01,
            -1.0579200e+02,  6.3060002e+00,  2.4511212e+07,
             9.9931999e+01, -1.0560500e+02,  7.0580001e+00,
             2.4511216e+07,  1.0022200e+02, -1.0525400e+02,
             4.9819999e+00,  2.4511220e+07,  1.0080000e+02,
            -1.0524700e+02,  5.0029998e+00,  2.4511224e+07,
             1.0034800e+02, -1.0541200e+02,  5.0700002e+00,
             2.4511228e+07,  1.0091700e+02, -1.0537800e+02,
             5.0890002e+00,  2.4511232e+07,  1.0095200e+02,
            -1.0513700e+02,  5.1250000e+00,  2.4511236e+07,
             1.0100900e+02, -1.0517800e+02,  5.0609999e+00,
             2.4511240e+07,  1.0154600e+02, -1.0510300e+02,
             5.1459999e+00,  2.4511244e+07,  1.0169500e+02,
            -1.0524300e+02,  5.1780000e+00,  2.4511248e+07,
             1.0185700e+02, -1.0529700e+

In [13]:
query_buffer[0:100]

PynqBuffer([-57.5588 ,  31.8779 ,   5.24626,   3.     , -54.2277 ,
             31.5253 ,   5.08681,   7.     , -54.2232 ,  32.0263 ,
              5.09999,  11.     , -53.6215 ,  33.1688 ,   5.10453,
             15.     , -52.8096 ,  33.1725 ,   5.06892,  19.     ,
            -50.4927 ,  33.2047 ,   4.96882,  23.     , -49.7397 ,
             33.1963 ,   4.93604,  27.     , -47.4575 ,  -6.14411,
              4.31842,  31.     , -47.2892 ,  -5.7782 ,   1.03069,
             35.     , -47.4269 ,  -5.61614,   2.66994,  39.     ,
            -47.3972 ,  -5.27702,   2.6688 ,  43.     , -47.4337 ,
             -5.80838,   4.31479,  47.     , -47.5608 ,  -5.62771,
              5.97389,  51.     , -47.5365 ,  -5.28835,   5.96795,
             55.     , -47.543  ,  -4.28357,   2.66959,  59.     ,
            -47.4504 ,  -4.80959,   4.3092 ,  63.     , -47.4378 ,
             -4.47011,   4.30661,  67.     , -47.5363 ,  -4.27582,
              5.95773,  71.     , -47.1376 ,  -3.10869,   1.03

In [14]:
query_buffer[29000:29200]

PynqBuffer([ 3.54129e+01,  5.55730e+00,  1.03374e+01,  2.90030e+04,
             3.54081e+01,  5.30264e+00,  1.03289e+01,  2.90070e+04,
             3.54402e+01,  6.60216e+00,  1.03783e+01,  2.90110e+04,
             3.54143e+01,  6.33874e+00,  1.03630e+01,  2.90150e+04,
             3.55055e+01,  7.65839e+00,  1.04350e+01,  2.90190e+04,
             3.54591e+01,  7.38435e+00,  1.04129e+01,  2.90230e+04,
             3.52146e+01,  2.36022e+01,  1.01946e+01,  2.90270e+04,
             3.52117e+01,  2.32457e+01,  1.01573e+01,  2.90310e+04,
             3.52366e+01,  2.46956e+01,  1.03138e+01,  2.90350e+04,
             3.52651e+01,  2.50781e+01,  1.03598e+01,  2.90390e+04,
             3.52721e+01,  3.36231e+01,  9.62469e+00,  2.90430e+04,
             3.56200e+01,  5.78245e+01,  1.02214e+01,  2.90470e+04,
             3.68540e+01, -8.63109e+01,  1.01758e+01,  2.90510e+04,
             3.70506e+01, -6.23470e+00,  7.99715e+00,  2.90550e+04,
             3.68870e+01,  2.18161e+01,  1.02978

In [15]:
data_index_buffer[0:100]

PynqBuffer([363, 364, 437, 505, 509, 521, 504, 422, 276, 282, 326, 336,
            377, 438, 453, 454, 464, 465, 466, 477, 498, 510, 402, 403,
            365, 401, 423, 478, 479, 128, 129, 134, 135, 167, 168, 169,
            196, 197, 198, 206, 255, 256, 270, 302, 337, 338, 366, 378,
            404, 424, 439,  90, 114, 124, 125, 130, 136, 137, 138, 144,
            158, 159, 160, 182, 288, 303, 304, 339,  99, 118, 139, 199,
            200, 271, 405, 407, 102, 119, 120, 145, 161, 170, 283, 346,
            406, 506, 511, 512, 516, 517, 526, 367, 409, 425, 310, 311,
            312, 328, 340, 342], dtype=int32)

In [16]:
data_index_buffer[6127800:6127900]

PynqBuffer([6127798, 6127801, 6127802, 6127803, 6127804, 6127806, 6127805,
            6127807, 6127808, 6127809, 6127810, 6127811, 6127812, 6127813,
            6127814, 6127815, 6127816, 6127817, 6127818, 6127824, 6127819,
            6127822, 6127823, 6127820, 6127821, 6127825, 6127826, 6127848,
            6127866, 6127859, 6127841, 6127846, 6127849, 6127851, 6127854,
            6127860, 6127842, 6127843, 6127847, 6127850, 6127838, 6127844,
            6127870, 6127845, 6127833, 6127839, 6127863, 6127867, 6127834,
            6127840, 6127852, 6127855, 6127856, 6127871, 6127835, 6127857,
            6127868, 6127872, 6127865, 6127861, 6127864, 6127869, 6127831,
            6127836, 6127830, 6127873, 6127853, 6127874, 6127858, 6127862,
            6127876, 6127878, 6127827, 6127832, 6127877, 6127879, 6127829,
            6127837, 6127828, 6127875], dtype=int32)

In [17]:
voxel_buffer[0:100]

PynqBuffer([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
            0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
            0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
            0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
            0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], dtype=int32)

In [18]:
voxel_buffer[1303300:1303420]

PynqBuffer([6127880, 6127880, 6127880, 6127880, 6127880, 6127880, 6127880,
            6127880, 6127880, 6127880, 6127880, 6127880, 6127880, 6127880,
            6127880, 6127880, 6127880, 6127880, 6127880, 6127880, 6127880,
            6127880, 6127880, 6127880, 6127880, 6127880, 6127880, 6127880,
            6127880, 6127880, 6127880, 6127880, 6127880, 6127880, 6127880,
            6127880, 6127880, 6127880, 6127880, 6127880, 6127880, 6127880,
            6127880, 6127880, 6127880, 6127880, 6127880, 6127880, 6127880,
            6127880, 6127880, 6127880, 6127880, 6127880, 6127880, 6127880,
            6127880, 6127880, 6127880, 6127880, 6127880, 6127880, 6127880,
            6127880, 6127880, 6127880, 6127880, 6127880, 6127880, 6127880,
            6127880, 6127880, 6127880, 6127880, 6127880, 6127880, 6127880,
            6127880, 6127880, 6127880, 6127880, 6127880, 6127880, 6127880,
            6127880, 6127880, 6127880, 6127880, 6127880, 6127880, 6127880,
            6127880, 6127

In [19]:
sub_buffer[0:100]

PynqBuffer([3093, 3095, 3096, 3096, 3097, 3112, 3120, 3144, 3154, 3290,
            3291, 3293, 3293, 3294, 3320, 3331, 3356, 3370, 3493, 3495,
            3497, 3498, 3499, 3527, 3539, 3562, 3570, 3705, 3705, 3705,
            3705, 3705, 3733, 3741, 3762, 3770, 3897, 3897, 3897, 3898,
            3899, 3921, 3929, 3952, 3958, 4051, 4052, 4053, 4053, 4054,
            4076, 4082, 4108, 4117, 4251, 4253, 4255, 4256, 4258, 4288,
            4295, 4324, 4327, 4455, 4455, 4457, 4457, 4459, 4492, 4499,
            4528, 4537, 4672, 4672, 4673, 4673, 4676, 4699, 4710, 4731,
            4740, 4889, 4889, 4891, 4891, 4894, 4923, 4934, 4965, 4980,
            5130, 5130, 5132, 5132, 5134, 5161, 5177, 5204, 5211, 5388],
           dtype=int32)

In [20]:
sub_buffer[179000:179100]

PynqBuffer([6124575, 6124575, 6124622, 6124639, 6124687, 6124713, 6124714,
            6124715, 6124719, 6124719, 6124721, 6124789, 6124860, 6124875,
            6124885, 6124885, 6124885, 6124885, 6124885, 6124885, 6124978,
            6125050, 6125060, 6125076, 6125076, 6125076, 6125076, 6125076,
            6125076, 6125184, 6125290, 6125369, 6125465, 6125476, 6125481,
            6125488, 6125488, 6125488, 6125605, 6125724, 6125830, 6125933,
            6125933, 6125934, 6125934, 6125936, 6125936, 6126036, 6126110,
            6126199, 6126267, 6126267, 6126270, 6126271, 6126273, 6126273,
            6126330, 6126358, 6126403, 6126425, 6126429, 6126429, 6126432,
            6126434, 6126489, 6126513, 6126549, 6126549, 6126549, 6126549,
            6126549, 6126549, 6126549, 6127190, 6127204, 6127211, 6127230,
            6127237, 6127247, 6127259, 6127267, 6127282, 6127880,       0,
                  0,       0,       0,       0,       0,       0,       0,
                  0,     

In [21]:
sub_flag_buffer[0:100]

PynqBuffer([1600000, 1600000, 1600000, 1600000, 1600000, 1600000, 1600000,
            1600000, 1600000, 1600000, 1600000, 1600000, 1600000, 1600000,
            1600000, 1600000, 1600000, 1600000, 1600000, 1600000, 1600000,
            1600000, 1600000, 1600000, 1600000, 1600000, 1600000, 1600000,
            1600000, 1600000, 1600000, 1600000, 1600000, 1600000, 1600000,
            1600000, 1600000, 1600000, 1600000, 1600000, 1600000, 1600000,
            1600000, 1600000, 1600000, 1600000, 1600000, 1600000, 1600000,
            1600000, 1600000, 1600000, 1600000, 1600000, 1600000, 1600000,
            1600000, 1600000, 1600000, 1600000, 1600000, 1600000, 1600000,
            1600000, 1600000, 1600000, 1600000, 1600000, 1600000, 1600000,
            1600000, 1600000, 1600000, 1600000, 1600000, 1600000, 1600000,
            1600000, 1600000, 1600000, 1600000, 1600000, 1600000, 1600000,
            1600000, 1600000, 1600000, 1600000, 1600000, 1600000, 1600000,
            1600000, 1600

In [22]:
sub_flag_buffer[1303300:1303420]

PynqBuffer([1600000, 1600000, 1600000, 1600000, 1600000, 1600000, 1600000,
            1600000, 1600000, 1600000, 1600000, 1600000, 1600000, 1600000,
            1600000, 1600000, 1600000, 1600000, 1600000, 1600000, 1600000,
            1600000, 1600000, 1600000, 1600000, 1600000, 1600000, 1600000,
            1600000, 1600000, 1600000, 1600000, 1600000, 1600000, 1600000,
            1600000, 1600000, 1600000, 1600000, 1600000, 1600000, 1600000,
            1600000, 1600000, 1600000, 1600000, 1600000, 1600000, 1600000,
            1600000, 1600000, 1600000, 1600000, 1600000, 1600000, 1600000,
            1600000, 1600000, 1600000, 1600000, 1600000, 1600000, 1600000,
            1600000, 1600000, 1600000, 1600000, 1600000, 1600000, 1600000,
            1600000, 1600000, 1600000, 1600000, 1600000, 1600000, 1600000,
            1600000, 1600000, 1600000, 1600000, 1600000, 1600000, 1600000,
            1600000, 1600000, 1600000, 1600000, 1600000, 1600000, 1600000,
            1600000, 1600

### 配置IP

我们可以直接使用IP的`write`方法，将刚分配的内存空间的地址写入到IP对应位置上

对于数据长度，我们可以直接在对应寄存器写入值。

In [23]:
overlay

In [24]:
icecs

In [25]:
data_len

24511520

In [26]:
query_len

29184

In [27]:
icecs.write(0x10,data_buffer.physical_address)
icecs.write(0x1c,6127880)
icecs.write(0x24,data_index_buffer.physical_address)
icecs.write(0x30,query_buffer.physical_address)
icecs.write(0x3c,7296)
icecs.write(0x5c,voxel_buffer.physical_address)
icecs.write(0x68,sub_flag_buffer.physical_address)
icecs.write(0x74,sub_buffer.physical_address)
icecs.write(0x44,result_buffer.physical_address)
icecs.write(0x50,nearest_buffer.physical_address)
#fir.s_axi_control.write(0x28,coef_buffer.physical_address)
#icecs.write(0x4c, 3829924)
#fir.write(0x4c, 957481)

### 启动IP

控制信号位于0x00地址，我们可以对其进行写入与读取来控制IP启动、监听是否完成。

In [28]:
import time

icecs.write(0x00, 0x01)
start_time = time.time()
while True:
    reg = icecs.read(0x00)
    #print(reg)
    if reg != 1:
        break
end_time = time.time()

print("耗时：{}s".format(end_time - start_time))

耗时：0.021071434020996094s


结果已经被写入到了`output_buffer`中，我们可以进行查看

In [29]:
result_buffer[0:200]

PynqBuffer([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
            0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
            0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
            0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
            0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
            0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
            0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
            0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
            0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
            0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], dtype=int32)

In [30]:
result_buffer[990:1010]

PynqBuffer([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
           dtype=int32)

In [31]:
result_buffer[15900:16010]

PynqBuffer([], dtype=int32)

In [32]:
result_buffer[7200:7300]

PynqBuffer([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
            0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
            0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
            0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
            0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], dtype=int32)

In [33]:
nearest_buffer[800:900]

PynqBuffer([100000., 100000., 100000., 100000., 100000., 100000., 100000.,
            100000., 100000., 100000., 100000., 100000., 100000., 100000.,
            100000., 100000., 100000., 100000., 100000., 100000., 100000.,
            100000., 100000., 100000., 100000., 100000., 100000., 100000.,
            100000., 100000., 100000., 100000., 100000., 100000., 100000.,
            100000., 100000., 100000., 100000., 100000., 100000., 100000.,
            100000., 100000., 100000., 100000., 100000., 100000., 100000.,
            100000., 100000., 100000., 100000., 100000., 100000., 100000.,
            100000., 100000., 100000., 100000., 100000., 100000., 100000.,
            100000., 100000., 100000., 100000., 100000., 100000., 100000.,
            100000., 100000., 100000., 100000., 100000., 100000., 100000.,
            100000., 100000., 100000., 100000., 100000., 100000., 100000.,
            100000., 100000., 100000., 100000., 100000., 100000., 100000.,
            100000., 1000

In [34]:
nearest_buffer[0:400]

PynqBuffer([100000., 100000., 100000., 100000., 100000., 100000., 100000.,
            100000., 100000., 100000., 100000., 100000., 100000., 100000.,
            100000., 100000., 100000., 100000., 100000., 100000., 100000.,
            100000., 100000., 100000., 100000., 100000., 100000., 100000.,
            100000., 100000., 100000., 100000., 100000., 100000., 100000.,
            100000., 100000., 100000., 100000., 100000., 100000., 100000.,
            100000., 100000., 100000., 100000., 100000., 100000., 100000.,
            100000., 100000., 100000., 100000., 100000., 100000., 100000.,
            100000., 100000., 100000., 100000., 100000., 100000., 100000.,
            100000., 100000., 100000., 100000., 100000., 100000., 100000.,
            100000., 100000., 100000., 100000., 100000., 100000., 100000.,
            100000., 100000., 100000., 100000., 100000., 100000., 100000.,
            100000., 100000., 100000., 100000., 100000., 100000., 100000.,
            100000., 1000

In [35]:
#result_buffer[15319600:15319700]

In [24]:
#result_buffer[7659800:7659900]